In [1]:
# Data processing
import numpy as np
import pandas as pd
import xarray as xr
import iris
import iris.pandas
import warnings
warnings.filterwarnings('ignore', module='iris')
from pathlib import Path
import dateutil.parser
from datetime import datetime, timedelta
# Visualization
import cartopy.util
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.dates import DateFormatter
from matplotlib.ticker import FuncFormatter
# Plotting parameters
plt.rcParams['mathtext.default'] = 'regular'
def m2km(x, pos):
    '''Convert meters to kilometers when plotting axis labels'''
    return int(x*1e-3) #'{:1.1f}'.format(x*1e-3)
savefig = False
# Species names
str_c2h6 = '$C_2H_6$'; str_c3h8 = '$C_3H_8$'
str_nc4h10 = '$nC_4H_{10}$'; str_ic4h10 = '$iC_4H_{10}$'
str_nc5h12 = '$nC_5H_{12}$'; str_ic5h12 = '$iC_5H_{12}$'
str_meono2 = '$MeONO_2$'; str_sbuono2 = '2-$BuONO_2$'
str_n2peono2 = '2-$PenONO_2$'; str_n3peono2 = '3-$PenONO_2$'; str_ipeono2 = '3-Methyl-2-$BuONO_2$'

#### Read data

In [2]:
path_to_atom = Path('../../../../../../obs/ATom/nc/data')
# Select flights
fname_atom_flight = 'MER-WAS_DC8_201701*.nc'
# Read ATom data
atom_dsinf = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), decode_times=True)
atom_dsmms = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), group='MMS', decode_cf=True)
atom_dswas = xr.open_mfdataset(sorted(path_to_atom.glob(fname_atom_flight)), group='WAS', decode_cf=True)

In [3]:
# Read precalculated UKCA trajectories
ukca_run_name = 'az166a'
compared_common_id = '170126_170130_1701_merge'
path_to_prcd = Path('../../../../processed') / ukca_run_name
traj_c2h6 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_c2h6.nc')
traj_c3h8 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_c3h8.nc')
traj_nc4h10 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_nc4h10.nc')
traj_ic4h10 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ic4h10.nc')
traj_nc5h12 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_nc5h12.nc')
traj_ic5h12 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ic5h12.nc')
traj_meono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_meono2.nc')
traj_sbuono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_sbuono2.nc')
traj_n2peono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_n2peono2.nc')
traj_n3peono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_n3peono2.nc')
traj_ipeono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ipeono2.nc')
# Load UKCA geospatial coordinates
ukca_lats = iris.load_cube(str(Path('../../../../data')/'ukca_coords_n96e_marcus_180705.nc'), 'mass_fraction_of_ozone_in_air').coord('latitude')
ukca_lons = iris.load_cube(str(Path('../../../../data')/'ukca_coords_n96e_marcus_180705.nc'), 'mass_fraction_of_ozone_in_air').coord('longitude')
ukca_alts = iris.load_cube(str(Path('../../../../data')/'ukca_coords_n96e_marcus_180705.nc'), 'mass_fraction_of_ozone_in_air').coord('altitude')

In [4]:
# Path to figures
path_to_figs = Path('../../../../../../results') / ukca_run_name / compared_common_id
path_to_figs.mkdir(exist_ok=True) # create a new folder if it doesn't exist
print(compared_common_id)

170126_170130_1701_merge


In [5]:
grid_lons, grid_lats = np.meshgrid(ukca_lons.points, ukca_lats.points)
sample_lats = atom_dsmms.G_LAT.data 
sample_lons = atom_dsmms.G_LONG.data+360
sample_alts = atom_dsmms.G_ALT.data

In [6]:
# Convert ATom C2H6, C2H4, C2H2, C3H8, C3H6 to pptvC to match UKCA lumped species, namely C2H6=C2H6+C2H4+C2H2, C3H8=C3H8+C3H6
atom_dswas_c2h6 = atom_dswas['Ethane_WAS']*2 + atom_dswas['Ethene_WAS']*2 + atom_dswas['Ethyne_WAS']*2
atom_dswas_c3h8 = atom_dswas['Propane_WAS']*3 + atom_dswas['Propene_WAS']*3
# Select model data where obs are present
traj_c2h6_ifobsexist = np.where(~np.isnan(atom_dswas_c2h6), traj_c2h6.data, atom_dswas_c2h6).squeeze()
traj_c3h8_ifobsexist = np.where(~np.isnan(atom_dswas_c3h8), traj_c3h8.data, atom_dswas_c3h8).squeeze()
traj_nc4h10_ifobsexist = np.where(~np.isnan(atom_dswas['n-Butane_WAS']), traj_nc4h10.data, atom_dswas['n-Butane_WAS']).squeeze()
traj_ic4h10_ifobsexist = np.where(~np.isnan(atom_dswas['i-Butane_WAS']), traj_ic4h10.data, atom_dswas['i-Butane_WAS']).squeeze()
traj_nc5h12_ifobsexist = np.where(~np.isnan(atom_dswas['n-Pentane_WAS']), traj_nc5h12.data, atom_dswas['n-Pentane_WAS']).squeeze()
traj_ic5h12_ifobsexist = np.where(~np.isnan(atom_dswas['i-Pentane_WAS']), traj_ic5h12.data, atom_dswas['i-Pentane_WAS']).squeeze()
traj_meono2_ifobsexist = np.where(~np.isnan(atom_dswas['MeONO2_WAS']), traj_meono2.data, atom_dswas['MeONO2_WAS']).squeeze()
traj_sbuono2_ifobsexist = np.where(~np.isnan(atom_dswas['2-BuONO2_WAS']), traj_sbuono2.data, atom_dswas['2-BuONO2_WAS']).squeeze()
traj_n2peono2_ifobsexist = np.where(~np.isnan(atom_dswas['2-PenONO2_WAS']), traj_n2peono2.data, atom_dswas['2-PenONO2_WAS']).squeeze()
traj_n3peono2_ifobsexist = np.where(~np.isnan(atom_dswas['3-PenONO2_WAS']), traj_n3peono2.data, atom_dswas['3-PenONO2_WAS']).squeeze()
traj_ipeono2_ifobsexist = np.where(~np.isnan(atom_dswas['3-Methyl-2-BuONO2_WAS']), traj_ipeono2.data, atom_dswas['3-Methyl-2-BuONO2_WAS']).squeeze()

In [7]:
# Convert xarray dataset with observational data to pandas dataframe
atom_c2h6 = atom_dswas_c2h6.to_dataframe(name='Lumped_C2H6_C2H4_C2H2')
atom_c3h8 = atom_dswas_c3h8.to_dataframe(name='Lumped_C3H8_C3H6')
atom_nc4h10 = atom_dswas['n-Butane_WAS'].to_dataframe()
atom_ic4h10 = atom_dswas['i-Butane_WAS'].to_dataframe()
atom_nc5h12 = atom_dswas['n-Pentane_WAS'].to_dataframe()
atom_ic5h12 = atom_dswas['i-Pentane_WAS'].to_dataframe()
atom_meono2 = atom_dswas['MeONO2_WAS'].to_dataframe()
atom_sbuono2 = atom_dswas['2-BuONO2_WAS'].to_dataframe()
atom_n2peono2 = atom_dswas['2-PenONO2_WAS'].to_dataframe()
atom_n3peono2 = atom_dswas['3-PenONO2_WAS'].to_dataframe()
atom_ipeono2 = atom_dswas['3-Methyl-2-BuONO2_WAS'].to_dataframe()

In [8]:
lons_meono2_notnan = np.where(atom_meono2['MeONO2_WAS'].isna()==False, sample_lons, atom_meono2['MeONO2_WAS'])
lats_meono2_notnan = np.where(atom_meono2['MeONO2_WAS'].isna()==False, sample_lats, atom_meono2['MeONO2_WAS'])

In [9]:
from shapely.geometry import Point, Polygon

In [16]:
# Define geographical region
corners = [(-160+360, 60), (-160+360, 75), (-145+360, 75), (-145+360, 60)]
region = Polygon(corners)

In [25]:
# Construct pairs of coordinate points
lons_lats = []
lons_lats_arr = []
for i, j in zip(lons_meono2_notnan, lats_meono2_notnan):
    lons_lats.append(Point(i, j))
    lons_lats_arr.append((i, j))

In [22]:
# Find points within the region
points_within = []
for p in lons_lats:
    points_within.append(p.within(region))

In [27]:
len(points_within)

273

In [28]:
lats_meono2_notnan.shape

(273,)

In [29]:
lats_meono2_notnan[points_within]

array([60.33771506, 60.99728171, 62.25365771, 63.45632663, 64.49820165,
       65.55266602, 66.62245971, 67.6358683 , 68.070925  , 68.50417282,
       68.85023085, 69.16533838, 69.49689894, 69.70570773, 69.90719711,
       70.18273622, 70.22881955, 70.20766952, 70.18993   , 70.17428341,
       70.26837791, 70.43590159, 70.69192925, 70.91101783, 70.97870047,
       71.03945757, 71.09865323, 71.15501897, 71.23260756, 71.264715  ,
       71.28471417, 71.27137758, 71.28532868, 71.28493971, 71.28420857,
       69.47642368, 67.13029711, 64.52694446, 61.82665517, 61.6757775 ,
       61.545766  , 61.41482359])

### without shapely

In [31]:
corners

[(200, 60), (200, 75), (215, 75), (215, 60)]

In [32]:
idx_within = ((lats_meono2_notnan <= 75)
              & (lats_meono2_notnan >= 60)
              & (lons_meono2_notnan >= 200)
              & (lons_meono2_notnan <= 215))

In [35]:
lats_meono2_notnan[idx_within]

array([60.33771506, 60.99728171, 62.25365771, 63.45632663, 64.49820165,
       65.55266602, 66.62245971, 67.6358683 , 68.070925  , 68.50417282,
       68.85023085, 69.16533838, 69.49689894, 69.70570773, 69.90719711,
       70.18273622, 70.22881955, 70.20766952, 70.18993   , 70.17428341,
       70.26837791, 70.43590159, 70.69192925, 70.91101783, 70.97870047,
       71.03945757, 71.09865323, 71.15501897, 71.23260756, 71.264715  ,
       71.28471417, 71.27137758, 71.28532868, 71.28493971, 71.28420857,
       69.47642368, 67.13029711, 64.52694446, 61.82665517, 61.6757775 ,
       61.545766  , 61.41482359])

In [ ]:
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
arr1 = np.where((-160+360 < lons_meono2_notnan) & (lons_meono2_notnan < -145+360))
arr1[0].shape

In [ ]:
arr2 = np.where((60 < lats_meono2_notnan) & (lats_meono2_notnan < 75))
arr2[0].shape

In [ ]:
np.where((arr1) & (arr2))

In [ ]:
# Plot sampling locations and model surface concentration (hybrid height level 0)
fig, ax = plt.subplots(figsize=(10,10), subplot_kw=dict(projection=ccrs.PlateCarree()), facecolor='w')
ax.scatter(lons_meono2_notnan, lats_meono2_notnan, transform=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.plot(grid_lons, grid_lats, color='b', alpha=0.7, transform=ccrs.PlateCarree());
ax.plot(grid_lons.T, grid_lats.T, color='b', alpha=0.7, transform=ccrs.PlateCarree());
ax.set_extent([-160, -145, 60, 75], crs=ccrs.PlateCarree())
# ax.set_extent([-160, -115, -2, 75], crs=ccrs.PlateCarree()) # both Jan flights

#### Time series of concentrations along flight track

In [ ]:
# Extract datetimes from ATom data
atom_datetime = atom_dsinf.time.values.astype('<M8[us]').astype(datetime)
atom_date_strt = atom_datetime[0]
atom_date_stop = atom_datetime[-1]
# Time series and cross section plotting parameters
if compared_common_id == '160801_160823_1608_merge':
    flght_separators = [147, 276, 427, 572, 735, 894, 1055, 1212, 1329] # manually select indices of flight start times in a merge
    slct_time_inds = np.arange(0, atom_dsinf.time.shape[0], 250) # manually choose frequency of datetimes for xticklabels
elif compared_common_id == '170126_170130_1701_merge':
    flght_separators = [141]
    slct_time_inds = np.arange(0, atom_dsinf.time.shape[0], 48)
elif compared_common_id == '170201_170221_1702_merge':
    flght_separators = [165, 324, 477, 633, 776, 928, 1077, 1236] 
    slct_time_inds = np.arange(0, atom_dsinf.time.shape[0], 250)
slct_time_lbls = [i.strftime('%H:%M\n%d-%m-%Y') for i in atom_dsinf.time.values[slct_time_inds].astype('<M8[us]').astype(datetime)]
print(slct_time_inds, atom_dsinf.time.shape)

In [ ]:
# Time series plotting parameters
def plot_ts(atom, ukca, species, ppn, picname):
    '''Plot observed and modelled species time series along a flight track (flight altitude is on the secondary y axis).'''
    fig, ax = plt.subplots(figsize=(12,4), facecolor='w')
    ax.plot(atom, color='k')
    ax.plot(ukca, color='r')
    ax2 = ax.twinx()
    ax2.plot(atom_dsmms.G_ALT, color='grey', linestyle='--', alpha=0.5)
    # common
    ax.set_title(species)
    ax.set_xlabel('Time, hours (UTC)')
    ax.set_ylabel(ppn)
    ax.set_xticks(slct_time_inds)
    ax.set_xticklabels(slct_time_lbls)
    ax.margins(x=0)
    ax.legend(['ATom', f'UKCA {ukca_run_name} montly mean'], frameon=False)
    ax2.yaxis.set_major_formatter(FuncFormatter(m2km))
    ax2.set_ylabel('Altitude, km', color='grey')
    ax2.tick_params('y', colors='grey')
    for x in flght_separators: ax.axvline(x=x, ymin=0, ymax=1500, color='grey', linestyle='--', linewidth=1)
    if savefig: fig.savefig(path_to_figs / f'{ukca_run_name}_{compared_common_id}_{picname}.png', bbox_inches='tight')

In [ ]:
plot_ts(atom_dswas_c2h6, traj_c2h6_ifobsexist*2, str_c2h6, 'pptvC', 'ts_RH_C2')

In [ ]:
plot_ts(atom_dswas_c3h8, traj_c3h8_ifobsexist*3, str_c3h8, 'pptvC', 'ts_RH_C3')

In [ ]:
plot_ts(atom_dswas['n-Butane_WAS'], traj_nc4h10_ifobsexist, str_nc4h10, 'pptv', 'ts_RH_C4n')

In [ ]:
plot_ts(atom_dswas['i-Butane_WAS'], traj_ic4h10_ifobsexist, str_ic4h10, 'pptv', 'ts_RH_C4i')

In [ ]:
plot_ts(atom_dswas['n-Pentane_WAS'], traj_nc5h12_ifobsexist, str_nc5h12, 'pptv', 'ts_RH_C5n')

In [ ]:
plot_ts(atom_dswas['i-Pentane_WAS'], traj_ic5h12_ifobsexist, str_ic5h12, 'pptv', 'ts_RH_C5i')

In [ ]:
plot_ts(atom_dswas['MeONO2_WAS'], traj_meono2_ifobsexist, str_meono2, 'pptv', 'ts_RONO2_C1')

In [ ]:
plot_ts(atom_dswas['2-BuONO2_WAS'], traj_sbuono2_ifobsexist, str_sbuono2, 'pptv', 'ts_RONO2_C4s')

In [ ]:
plot_ts(atom_dswas['2-PenONO2_WAS'], traj_n2peono2_ifobsexist, str_n2peono2, 'pptv', 'ts_RONO2_C5n2')

In [ ]:
plot_ts(atom_dswas['3-PenONO2_WAS'], traj_n3peono2_ifobsexist, str_n3peono2, 'pptv', 'ts_RONO2_C5n3')

In [ ]:
plot_ts(atom_dswas['3-Methyl-2-BuONO2_WAS'], traj_ipeono2_ifobsexist, str_ipeono2, 'pptv', 'ts_RONO2_C5i')

#### Cross section along flight track

In [ ]:
# Read precalculated UKCA 2D trajectories
crsc_c2h6 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_c2h6.nc')
crsc_c3h8 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_c3h8.nc')
crsc_nc4h10 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_nc4h10.nc')
crsc_ic4h10 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ic4h10.nc')
crsc_nc5h12 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_nc5h12.nc')
crsc_ic5h12 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ic5h12.nc')
crsc_meono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_meono2.nc')
crsc_sbuono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_sbuono2.nc')
crsc_n2peono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_n2peono2.nc')
crsc_n3peono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_n3peono2.nc')
crsc_ipeono2 = iris.load_cube(str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ipeono2.nc')

In [ ]:
# Convert xarray dataset with observational data to pandas dataframe
atom_c2h6 = atom_dswas_c2h6.to_dataframe(name='Lumped_C2H6_C2H4_C2H2')
atom_c3h8 = atom_dswas_c3h8.to_dataframe(name='Lumped_C3H8_C3H6')
atom_nc4h10 = atom_dswas['n-Butane_WAS'].to_dataframe()
atom_ic4h10 = atom_dswas['i-Butane_WAS'].to_dataframe()
atom_nc5h12 = atom_dswas['n-Pentane_WAS'].to_dataframe()
atom_ic5h12 = atom_dswas['i-Pentane_WAS'].to_dataframe()
atom_meono2 = atom_dswas['MeONO2_WAS'].to_dataframe()
atom_sbuono2 = atom_dswas['2-BuONO2_WAS'].to_dataframe()
atom_n2peono2 = atom_dswas['2-PenONO2_WAS'].to_dataframe()
atom_n3peono2 = atom_dswas['3-PenONO2_WAS'].to_dataframe()
atom_ipeono2 = atom_dswas['3-Methyl-2-BuONO2_WAS'].to_dataframe()

In [ ]:
# Find max between maxima and max between minima in ATom and model cross section data for plotting a cross section with two colormaps
cs_c2h6_max = max(np.nanmax(atom_c2h6.values), np.nanmax(crsc_c2h6.data*2))
cs_c3h8_max = max(np.nanmax(atom_c3h8.values), np.nanmax(crsc_c3h8.data*3))
cs_nc4h10_max = max(np.nanmax(atom_nc4h10.values), np.nanmax(crsc_nc4h10.data))
cs_ic4h10_max = max(np.nanmax(atom_ic4h10.values), np.nanmax(crsc_ic4h10.data))
cs_nc5h12_max = max(np.nanmax(atom_nc5h12.values), np.nanmax(crsc_nc5h12.data))
cs_ic5h12_max = max(np.nanmax(atom_ic5h12.values), np.nanmax(crsc_ic5h12.data))
cs_meono2_max = max(np.nanmax(atom_meono2.values), np.nanmax(crsc_meono2.data))
cs_sbuono2_max = max(np.nanmax(atom_sbuono2.values), np.nanmax(crsc_sbuono2.data))
cs_n2peono2_max = max(np.nanmax(atom_n2peono2.values), np.nanmax(crsc_n2peono2.data))
cs_n3peono2_max = max(np.nanmax(atom_n3peono2.values), np.nanmax(crsc_n3peono2.data))
cs_ipeono2_max = max(np.nanmax(atom_ipeono2.values), np.nanmax(crsc_ipeono2.data))

cs_c2h6_min = min(np.nanmax(atom_c2h6.values), np.nanmax(crsc_c2h6.data*2))
cs_c3h8_min = min(np.nanmax(atom_c3h8.values), np.nanmax(crsc_c3h8.data*3))
cs_nc4h10_min = min(np.nanmax(atom_nc4h10.values), np.nanmax(crsc_nc4h10.data))
cs_ic4h10_min = min(np.nanmax(atom_ic4h10.values), np.nanmax(crsc_ic4h10.data))
cs_nc5h12_min = min(np.nanmax(atom_nc5h12.values), np.nanmax(crsc_nc5h12.data))
cs_ic5h12_min = min(np.nanmax(atom_ic5h12.values), np.nanmax(crsc_ic5h12.data))
cs_meono2_min = min(np.nanmax(atom_meono2.values), np.nanmax(crsc_meono2.data))
cs_sbuono2_min = min(np.nanmax(atom_sbuono2.values), np.nanmax(crsc_sbuono2.data))
cs_n2peono2_min = min(np.nanmax(atom_n2peono2.values), np.nanmax(crsc_n2peono2.data))
cs_n3peono2_min = min(np.nanmax(atom_n3peono2.values), np.nanmax(crsc_n3peono2.data))
cs_ipeono2_min = min(np.nanmax(atom_ipeono2.values), np.nanmax(crsc_ipeono2.data))

In [ ]:
# Cross section plotting parameters
mrgd_time_inds = np.arange(0,atom_dsinf.time.values.shape[0],1) # merged time indices
# Merge two colormaps: color concentrations present in obs and model using one colormap and obs higher than model using another colormap
class MidpointNormalize(colors.Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)
    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
colors_low = plt.cm.Blues(np.linspace(0,1,128))
colors_high = plt.cm.autumn_r(np.linspace(0,1,128))
colors_stack = np.vstack((colors_low, colors_high))
mrgd_cmap = colors.LinearSegmentedColormap.from_list('merged_cmap', colors_stack)
cs_pc_kwargs = dict(cmap=mrgd_cmap, vmin=0) # model data pcolormesh options
cs_sc_kwargs = dict(cmap=mrgd_cmap, vmin=0, edgecolors='k', linewidth=0.2) # obs data scatter options
def plot_cs(atom, ukca, species, ppn, picname, cs_min, cs_max):
    '''Plot a cross section along a flight track, modelled data as background and observed data as dots.'''
    fig, ax = plt.subplots(figsize=(14,5), facecolor='w')
    p0 = ax.pcolormesh(mrgd_time_inds, crsc_c2h6.coord('altitude').points, ukca, norm=MidpointNormalize(midpoint=cs_min), vmax=cs_max, **cs_pc_kwargs)
    ax.scatter(mrgd_time_inds, atom_dsmms.G_ALT, c=atom.squeeze(), norm=MidpointNormalize(midpoint=cs_min), vmax=cs_max, **cs_sc_kwargs)
    fig.colorbar(p0, label=ppn)
    # common
    ax.set_title(species, loc='center')
    ax.set_title(f'ATom vs UKCA {ukca_run_name} montly mean', loc='right')
    ax.set_xlabel('Time, hours (UTC)')
    ax.set_ylabel('Altitude, km')
    ax.set_ylim(0,13500)
    ax.set_xticks(slct_time_inds)
    ax.set_xticklabels(slct_time_lbls)
    ax.yaxis.set_major_formatter(FuncFormatter(m2km))
    ax.margins(x=0)
    for x in flght_separators: ax.axvline(x=x, ymin=0, ymax=13500, color='grey', linestyle='--', linewidth=1)
    if savefig: fig.savefig(path_to_figs / f'{ukca_run_name}_{compared_common_id}_{picname}.png', bbox_inches='tight')

In [ ]:
plot_cs(atom_c2h6, crsc_c2h6.data*2, str_c2h6, 'pptvC', 'cs_RH_C2', cs_c2h6_min, cs_c2h6_max)

In [ ]:
plot_cs(atom_c3h8, crsc_c3h8.data*3, str_c3h8, 'pptvC', 'cs_RH_C3', cs_c3h8_min, cs_c3h8_max)

In [ ]:
plot_cs(atom_nc4h10, crsc_nc4h10.data, str_nc4h10, 'pptv', 'cs_RH_C4n', cs_nc4h10_min, cs_nc4h10_max)

In [ ]:
plot_cs(atom_ic4h10, crsc_ic4h10.data, str_ic4h10, 'pptv', 'cs_RH_C4i', cs_ic4h10_min, cs_ic4h10_max)

In [ ]:
plot_cs(atom_nc5h12, crsc_nc5h12.data, str_nc5h12, 'pptv', 'cs_RH_C5n', cs_nc5h12_min, cs_nc5h12_max)

In [ ]:
plot_cs(atom_ic5h12, crsc_ic5h12.data, str_ic5h12, 'pptv', 'cs_RH_C5i', cs_ic5h12_min, cs_ic5h12_max)

In [ ]:
plot_cs(atom_meono2, crsc_meono2.data, str_meono2, 'pptv', 'cs_RONO2_C1', cs_meono2_min, cs_meono2_max)

In [ ]:
plot_cs(atom_sbuono2, crsc_sbuono2.data, str_sbuono2, 'pptv', 'cs_RONO2_C4s', cs_sbuono2_min, cs_sbuono2_max)

In [ ]:
plot_cs(atom_n2peono2, crsc_n2peono2.data, str_n2peono2, 'pptv', 'cs_RONO2_C5n2', cs_n2peono2_min, cs_n2peono2_max)

In [ ]:
plot_cs(atom_n3peono2, crsc_n3peono2.data, str_n3peono2, 'pptv', 'cs_RONO2_C5n3', cs_n3peono2_min, cs_n3peono2_max)

In [ ]:
plot_cs(atom_ipeono2, crsc_ipeono2.data, str_ipeono2, 'pptv', 'cs_RONO2_C5i', cs_ipeono2_min, cs_ipeono2_max)

#### Scatter plot data along flight track

In [ ]:
# Find max concentration between observational and model trajectory data
sct_max_c2h6 = max(np.nanmax(atom_c2h6.values), np.nanmax(traj_c2h6.data*2))
sct_max_c3h8 = max(np.nanmax(atom_c3h8.values), np.nanmax(traj_c3h8.data*3))
sct_max_nc4h10 = max(np.nanmax(atom_nc4h10.values), np.nanmax(traj_nc4h10.data))
sct_max_ic4h10 = max(np.nanmax(atom_ic4h10.values), np.nanmax(traj_ic4h10.data))
sct_max_nc5h12 = max(np.nanmax(atom_nc5h12.values), np.nanmax(traj_nc5h12.data))
sct_max_ic5h12 = max(np.nanmax(atom_ic5h12.values), np.nanmax(traj_ic5h12.data))
sct_max_meono2 = max(np.nanmax(atom_meono2.values), np.nanmax(traj_meono2.data))
sct_max_sbuono2 = max(np.nanmax(atom_sbuono2.values), np.nanmax(traj_sbuono2.data))
sct_max_n2peono2 = max(np.nanmax(atom_n2peono2.values), np.nanmax(traj_n2peono2.data))
sct_max_n3peono2 = max(np.nanmax(atom_n3peono2.values), np.nanmax(traj_n3peono2.data))
sct_max_ipeono2 = max(np.nanmax(atom_ipeono2.values), np.nanmax(traj_ipeono2.data))

In [ ]:
def plot_one2one(atom, ukca, species, ppn, picname, sct_max):
    '''Plot all flight track data as a scatter plot.'''
    fig, ax = plt.subplots(figsize=(6,5), facecolor='w')
    p0 = ax.scatter(atom, ukca, c=atom_dsmms.G_ALT)
    ax.plot([0, sct_max], [0, sct_max], color='k', linestyle='--') # 'one-to-one' line
    ax.set_title(f'{species}, {ppn}', loc='right')
    ax.set_title(f'{atom_date_strt.strftime("%-d")}-{atom_date_stop.strftime("%-d %b %Y")}', loc='left')
    ax.set_xlabel('ATom')
    ax.set_ylabel(f'UKCA {ukca_run_name} montly mean')
    ax.set_xlim(0, sct_max)
    ax.set_ylim(0, sct_max)
    fig.colorbar(p0, label='Altitude, km', format=FuncFormatter(m2km))
    if savefig: fig.savefig(path_to_figs / f'{ukca_run_name}_{compared_common_id}_{picname}.png', bbox_inches='tight')

In [ ]:
plot_one2one(atom_c2h6, traj_c2h6.data*2, str_c2h6, 'pptvC', '1to1_RH_C2', sct_max_c2h6)

In [ ]:
plot_one2one(atom_c3h8, traj_c3h8.data*3, str_c3h8, 'pptvC', '1to1_RH_C3', sct_max_c3h8)

In [ ]:
plot_one2one(atom_nc4h10, traj_nc4h10.data, str_nc4h10, 'pptv', '1to1_RH_C4n', sct_max_nc4h10)

In [ ]:
plot_one2one(atom_ic4h10, traj_ic4h10.data, str_ic4h10, 'pptv', '1to1_RH_C4i', sct_max_ic4h10)

In [ ]:
plot_one2one(atom_nc5h12, traj_nc5h12.data, str_nc5h12, 'pptv', '1to1_RH_C5n', sct_max_nc5h12)

In [ ]:
plot_one2one(atom_ic5h12, traj_ic5h12.data, str_ic5h12, 'pptv', '1to1_RH_C5i', sct_max_ic5h12)

In [ ]:
plot_one2one(atom_meono2, traj_meono2.data, str_meono2, 'pptv', '1to1_RONO2_C1', sct_max_meono2)

In [ ]:
plot_one2one(atom_sbuono2, traj_sbuono2.data, str_sbuono2, 'pptv', '1to1_RONO2_C4s', sct_max_sbuono2)

In [ ]:
plot_one2one(atom_n2peono2, traj_n2peono2.data, str_n2peono2, 'pptv', '1to1_RONO2_C5n2', sct_max_n2peono2)

In [ ]:
plot_one2one(atom_n3peono2, traj_n3peono2.data, str_n3peono2, 'pptv', '1to1_RONO2_C5n3', sct_max_n3peono2)

In [ ]:
plot_one2one(atom_ipeono2, traj_ipeono2.data, str_ipeono2, 'pptv', '1to1_RONO2_C5i', sct_max_ipeono2)